<p style="color: red; font-weight: bold; font-size: xx-large; text-align: center;" align="center">
    'CARA' (COVID Airborne Risk Assessment)
</p>

<p style="color: blue; font-weight: bold; font-size: x-large; text-align: center;" align="center">
Airborne Transmission of SARS-CoV-2
</p>

In [ ]:
import cara.apps

app = cara.apps.ExpertApplication()
app.widget